# Notebook reading VisiumHD Dataset of Mouse Lung
- Dataset: https://www.10xgenomics.com/datasets/visium-hd-cytassist-gene-expression-mouse-lung-fresh-frozen
- Created by: Christian Eger
- Created on: 5th of November 2024
- Environment: environments/squidpy-env

Loading VisiumHD datasets mapped with SpaceRanger can be achieved with spatialdata_io.visiumhd. To convert the dataset to the anndata format expected for NicheCompass we additionally need to run to_legacy_anndata.

## Setup

### Import modules

In [8]:
from pathlib import Path
import spatialdata_io
from spatialdata_io.converters import legacy_anndata
from datetime import datetime

### Setup paths

In [9]:
notebook_path = Path.cwd() / "0-Loading_raw_data.ipynb"
notebook_execution_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
repo_path = Path.cwd().parent
data_folder_path = repo_path / ".data"

## Processing

In [10]:
spot_resolution = "square_002um"
images_resolution = "downscaled_hires"
sdata_path = data_folder_path / "raw" / "VisiumHD_Mouse_Lung" 
sdata = spatialdata_io.visium_hd(sdata_path)
adata = legacy_anndata.to_legacy_anndata(sdata, coordinate_system=images_resolution, table_name=spot_resolution, include_images=True)
adata.var_names_make_unique()
adata

/home/ceger/miniforge3/envs/squidpy-env/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/ceger/miniforge3/envs/squidpy-env/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/ceger/miniforge3/envs/squidpy-env/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_731815/840793048.py:4: UserWarning: No full resolution image found. If incorrect, please specify the path in the `fullres_image_file` parameter when calling the `visium_hd` reader function.
  sdata = spatialdata_io.visium_hd(sdata_path)
/home/ceger/miniforge3/envs/squidpy-env/lib/pytho

## Export

### Create Processing logs

In [ ]:
adata.uns["log"] = {
    "dataset_name": "VisiumHD_mouse_lung"
}

In [ ]:
logging_message = f"\
VisiumHD mouse lung data was downloaded from https://www.10xgenomics.com/datasets/visium-hd-cytassist-gene-expression-mouse-lung-fresh-frozen, \
read via spatialdata_io.visiumhd and converted to an anndata object at resolution {spot_resolution}\
"
output_path = data_folder_path / "processed" / "preprocessing" / f'{notebook_execution_time}_{adata.uns["log"]["dataset_name"]}_{spot_resolution}.h5ad'
adata.uns["log"][notebook_execution_time] = {
    "script": str(notebook_path),
    "logging_message": logging_message,
    "input_file": str(sdata_path),
    "output_file": str(output_path),
}

### Save file

In [ ]:
adata.write_h5ad(adata.uns["log"][notebook_execution_time]["output_file"])